In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [217]:
df_products_detailed = pd.read_csv("products_detailed.csv")
df_products_detailed.head()

,id,name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,is_natural_organic,is_natural_sephora,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_name,brand_id,brand_long
0,P420666,Water Pocket Sheet Mask Sleeping Mask (Replenishing),"A comfortable, non-slip mask that forms a moisture wrap, recharging and refreshing tired skin.","Which skin type is it good for? Normal Oily Combination Dry Sensitive What it is: A comfortable, non-slip mask that forms a moisture wrap, recharging and refreshing tired skin. Solutions for: - Dryness - Dullness and uneven texture - Uneven skintone If you want to know more… Reveal a more beautiful complexion with this replenishing mask. Through LANEIGE’s exclusive Moisture Wrap technology, a breathable film locks in hydration and reduces the look of tired skin. The comfortable, non-slip mask features diamond-shaped water pockets that releases essence slowly, for long-lasting active-ingredient impact. What it is formulated WITHOUT: - Parabens - Phthalates What else you need to know: This product is dermatologist tested. LANEIGE does not test ingredients or products on animals.",1969427,1 single-use mask,6.00,None,NaN,6.00,6.00,0,0,0,4.47,32,8235,"{'cat1440040': 'Sheet Masks', 'cat1440038': 'Masks', 'cat150006': 'Skincare'}",https://www.sephora.com/product/water-pocket-sheet-mask-sleeping-mask-replenishing-P420666?skuId=1969427,https://www.sephora.com/productimages/sku/s1969427-main-zoom.jpg,"Water, Dipropylene Glycol, Glycerin, 1,2-Hexanediol, Phenoxyethanol, Carbomer, Tromethamine, Bis-PEG-18 Methyl Ether Dimethyl Silane, PEG-60 Hydrogenated Castor Oil, Ethylhexylglycerin, Adenosine, Xanthan Gum, Fragrance, Disodium EDTA, Malpighia Emarginata (Acerola) Fruit Extract, Euterpe Oleracea Fruit Extract, Propanediol, 1, 2-Hexanediol, Ethylhexylglycerin, Sodium Magnesium Silicate, Disodium EDTA, Glutathione.","Suggested Usage: -1. Apply after toner. Open the pouch and remove the sheet mask. -2. Gently apply the mask onto face and remove the protective film. -3. Leave on for 20 minutes and remove. -4. Gently pat to absorb essence. Precautions: -1. Stop using the product immediately and consult a dermatologist when redness, swelling, or other skin irritations occur. -2. Avoid applying the product on open wounds or skin inflammation. -3. Keep away from direct sunlight and out of reach of infants and children.",LANEIGE,6125,"In the Laneige labs, we’ve spent over 25 years studying Water Science. Why water, you ask? Well, we’re a skin hydration brand! As the true Hydration Experts, we created Advanced Water Complexes dedicated to create smarter skincare solutions for you. So drink up and hydrate with Laneige!"
1,P448857,Apple Cider Vinegar Detangling Conditioner,"An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine.","What it is: An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine. Hair Type: Straight, Wavy, Curly, and Coily Hair Texture: Fine, Medium, and Thick Key Benefits: Detangles, Adds Shine, and Prevents Damage Formulation: Liquid Highlighted Ingredients: - Sunflower Seed Extract: Protects hair color. - Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Ingredient Callouts: Free of sulfates SLS and SLES, parabens, and phthalates. This product is also vegan, cruelty-free, and gluten-free, and comes in recyclable packaging. What Else You Need to Know: This conditioner is formulated to detangle hair and provide an extra conditioning boost after cleansing hair. It is infused with conditioning agents argan oil, aloe vera, and lavender extract to nurture hair and sunflower seed extract to protect hair color.",2276269,6.5 oz/ 192 mL,30.00,None,NaN,30.00,30.00,0,0,0,2.80,5,280,"{'cat60131'

In [218]:
df_products_detailed.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5734 entries, 0 to 5733
Data columns (total 25 columns):
id                    5734 non-null object
name                  5734 non-null object
short_desc            5734 non-null object
long_desc             5734 non-null object
item                  5734 non-null int64
item_name             4975 non-null object
list_price            5734 non-null float64
variation             5734 non-null object
variants              2570 non-null object
price_low             5734 non-null float64
price_high            5734 non-null float64
is_limited_edition    5734 non-null int64
is_natural_organic    5734 non-null int64
is_natural_sephora    5734 non-null int64
rating                5627 non-null float64
review_count          5734 non-null int64
user_favorites        5734 non-null int64
categories            5730 non-null object
url                   5734 non-null object
image_url             5734 non-null object
ingredients           4820 non-null

In [219]:
categories = []
for group in df_products_detailed[df_products_detailed.categories.isna()==False].categories.unique():
    group = json.loads(group.replace("\'", "\""))
    for key, val in group.items():
        if key:
            categories.append(val)
            
categories = list(set(categories))
len(categories), categories

(157,
 ['Hair Styling & Treatments',
  'Eye Creams & Treatments',
  'Cologne',
  'Mini Size',
  'Lip Balm & Treatment',
  'Face Oils',
  'Perfume',
  'Scalp & Hair Treatments',
  'Eyeshadow',
  'Treatments',
  'Tinted Moisturizer',
  'Hair Removal',
  'Mists & Essences',
  'BB & CC Cream',
  'Candles & Home Scents',
  'Hair Removal & Shaving',
  'Blemish & Acne Treatments',
  'Facial Cleansing Brushes',
  'Hair Straighteners & Flat Irons',
  'Face Wash & Cleansers',
  'Deodorant & Antiperspirant',
  'Scrub & Exfoliants',
  'Hair Primers',
  'Eyelash Curlers',
  'Body Care',
  'Curling Irons',
  'Curls & Coils',
  'Face Primer',
  'Face Sets',
  'Best For',
  'Sunscreen',
  'Anti-Aging',
  'Lip Plumper',
  'Makeup',
  'Conditioner',
  'Hair Masks',
  'False Eyelashes',
  'Mascara',
  'Color Correct',
  'Eye Sets',
  'Teeth Whitening',
  'Lip Liner',
  'Bath & Body',
  'Concealer',
  'Cologne Gift Sets',
  'Holistic Wellness',
  'Face Wash',
  'Contour',
  'Sheet Masks',
  'Eye',
  'High

In [220]:
df_category = df_products_detailed[["id", "categories"]].copy()

for i in range(len(df_category)):
    if df_category.categories.iloc[i] is np.nan:
        continue
    else:
        categories = json.loads(df_category.categories.iloc[i].replace("\'", "\""))
        for cat in categories.values():
            header=cat.lower().replace(" ", "_").replace("&", "and").replace("-", "_")
            df_category.at[i, header] = 1
        
display(list(df_category.columns))

['id',
 'categories',
 'sheet_masks',
 'masks',
 'skincare',
 'conditioner',
 'shampoo_and_conditioner',
 'hair',
 'mini_size',
 'concealer',
 'face',
 'makeup',
 'rollerballs_and_travel_size',
 'women',
 'fragrance',
 'value_and_gift_sets',
 'face_masks',
 'for_body',
 'self_tanners',
 'bath_and_body',
 'moisturizers',
 'perfume',
 'hair_styling_products',
 'hair_styling_and_treatments',
 'face_wash_and_cleansers',
 'cleansers',
 'lip_gloss',
 'lip',
 'hair_straighteners_and_flat_irons',
 'hair_tools',
 'blush',
 'cheek',
 'lotions_and_oils',
 'bath_and_shower',
 'men',
 'facial_cleansing_brushes',
 'high_tech_tools',
 'cellulite_and_stretch_marks',
 'body_care',
 'shampoo',
 'eye_primer',
 'eye',
 'lipstick',
 'hair_masks',
 'perfume_gift_sets',
 'body_lotions_and_body_oils',
 'body_moisturizers',
 'holistic_wellness',
 'wellness',
 'face_primer',
 'eye_palettes',
 'false_eyelashes',
 'body_wash_and_shower_gel',
 'lip_balm_and_treatment',
 'scrub_and_exfoliants',
 'eyebrow',
 'eyelin

In [191]:
df_category.drop(columns="categories", axis=1, inplace=True)
df_category.fillna(0, inplace=True)

In [192]:
for col in df_category.columns[1:]:
    df_category[col] = df_category[col].astype("int64")
df_category  

,id,sheet_masks,masks,skincare,conditioner,shampoo_and_conditioner,hair,mini_size,concealer,face,makeup,rollerballs_and_travel_size,women,fragrance,value_and_gift_sets,face_masks,for_body,self_tanners,bath_and_body,moisturizers,perfume,hair_styling_products,hair_styling_and_treatments,face_wash_and_cleansers,cleansers,lip_gloss,lip,hair_straighteners_and_flat_irons,hair_tools,blush,cheek,lotions_and_oils,bath_and_shower,men,facial_cleansing_brushes,high_tech_tools,cellulite_and_stretch_marks,body_care,shampoo,eye_primer,eye,lipstick,hair_masks,perfume_gift_sets,body_lotions_and_body_oils,body_moisturizers,holistic_wellness,wellness,face_primer,eye_palettes,false_eyelashes,body_wash_and_shower_gel,lip_balm_and_treatment,scrub_and_exfoliants,eyebrow,eyeliner,exfoliators,skincare_sets,face_brushes,brushes_and_applicators,tools_and_brushes,face_serums,treatments,beauty_supplements,lip_sets,nail,tinted_moisturizer,highlighter,face_sets,makeup_removers,eye_creams_and_treatments,eye_care,face_wipes,hair_brushes_and_combs,leave_in_conditioner,mascara,foundation,body_products,other_needs,shaving,bb_and_cc_cream,hair_oil,cologne_gift_sets,dry_shampoo,bronzer,face_sunscreen,sun_care,liquid_lipstick,hair_accessories,lip_liner,cologne,lip_balms_and_treatments,lip_treatments,deodorant_and_antiperspirant,tweezers_and_eyebrow_tools,accessories,anti_aging,hair_primers,facial_peels,brush_cleaners,curling_irons,hand_cream_and_foot_cream,brush_sets,scalp_and_hair_treatments,blemish_and_acne_treatments,eye_cream,toners,sponges_and_applicators,setting_spray_and_powder,moisturizer_and_treatments,face_oils,eye_brushes,facial_rollers,face_wash,candles,candles_and_home_scents,body_mist_and_hair_mist,makeup_palettes,value_sets,mists_and_essences,color_correct,eyeshadow,body_sprays_and_deodorant,aftershave,hair_dryers,hair_spray,contour,makeup_brushes_and_applicators,eye_sets,body_sunscreen,eyelash_curlers,night_creams,decollete_and_neck_creams,sunscreen,mirrors_and_sharpeners,eye_masks,hair_thinning_and_hair_loss,teeth_whitening,lip_brushes,cheek_palettes,bath_soaks_and_bubble_bath,makeup_bags_and_travel_cases,for_face,blotting_papers,color_care,best_for,hair_products,bb_and_cc_creams,lip_plumper,lip_stain,cleansing_brushes,deodorant_for_men,hand_wash_and_soap,hair_removal,diffusers,hair_removal_and_shaving,curls_and_coils,makeup_and_travel_cases
0,P420666,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,P448857,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,P425464,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,P428606,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,P391656,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,..

In [8]:
df_category.to_csv("category.csv", index=False)

In [146]:
df_category = pd.read_csv("category.csv")
df_category

,id,conditioner,shampoo_and_conditioner,hair,mini_size,concealer,face,makeup,value_and_gift_sets,skincare,moisturizers,rollerballs_and_travel_size,women,fragrance,blush,cheek,bath_and_shower,men,facial_cleansing_brushes,high_tech_tools,perfume,cellulite_and_stretch_marks,body_care,bath_and_body,shampoo,eye_primer,eye,lipstick,lip,perfume_gift_sets,face_primer,false_eyelashes,body_wash_and_shower_gel,hair_styling_products,hair_styling_and_treatments,lip_gloss,scrub_and_exfoliants,face_masks,masks,eyeliner,skincare_sets,face_brushes,brushes_and_applicators,hair_straighteners_and_flat_irons,hair_tools,tools_and_brushes,beauty_supplements,wellness,lip_sets,face_wash_and_cleansers,cleansers,face_sets,eye_creams_and_treatments,eye_care,highlighter,lip_balm_and_treatment,for_body,self_tanners,leave_in_conditioner,mascara,foundation,nail,body_products,other_needs,shaving,bb_and_cc_cream,hair_oil,dry_shampoo,lotions_and_oils,bronzer,face_serums,treatments,hair_masks,face_sunscreen,sun_care,liquid_lipstick,hair_accessories,lip_liner,cologne,tinted_moisturizer,eye_palettes,deodorant_and_antiperspirant,hair_brushes_and_combs,anti_aging,hair_primers,facial_peels,body_lotions_and_body_oils,body_moisturizers,brush_cleaners,curling_irons,hand_cream_and_foot_cream,brush_sets,exfoliators,scalp_and_hair_treatments,holistic_wellness,blemish_and_acne_treatments,eye_cream,toners,sponges_and_applicators,setting_spray_and_powder,moisturizer_and_treatments,face_oils,facial_rollers,lip_balms_and_treatments,lip_treatments,sheet_masks,candles,candles_and_home_scents,value_sets,face_wash,eyebrow,tweezers_and_eyebrow_tools,accessories,eyeshadow,color_correct,aftershave,hair_spray,contour,eye_brushes,makeup_brushes_and_applicators,makeup_removers,night_creams,decollete_and_neck_creams,mists_and_essences,sunscreen,mirrors_and_sharpeners,eye_masks,body_mist_and_hair_mist,hair_thinning_and_hair_loss,cologne_gift_sets,hair_dryers,lip_brushes,teeth_whitening,bath_soaks_and_bubble_bath,makeup_bags_and_travel_cases,for_face,makeup_palettes,color_care,best_for,cheek_palettes,hair_products,body_sunscreen,lip_plumper,bb_and_cc_creams,body_sprays_and_deodorant,cleansing_brushes,eye_sets,deodorant_for_men,hand_wash_and_soap,lip_stain,hair_removal,diffusers,face_wipes,hair_removal_and_shaving,curls_and_coils,eyelash_curlers,makeup_and_travel_cases
0,P448857,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,P425464,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,P428606,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,P458504,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,P442109,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,

In [208]:
urls = [
    "https://www.sephora.com/beauty/new-beauty-products",
    "https://www.sephora.com/shop/gifts-for-her",
    "https://www.sephora.com/shop/gifts-for-men",
    "https://www.sephora.com/shop/gifts-for-them",
    "https://www.sephora.com/shop/gifts-for-teenage-girls",
    "https://www.sephora.com/shop/gift-sets-for-men",
    "https://www.sephora.com/shop/travel-size-toiletries",
    "https://www.sephora.com/shop/value-sets",
    "https://www.sephora.com/shop/editors-picks-gifts?",
    "https://www.sephora.com/shop/luxury-gifts",
    "https://www.sephora.com/shop/exclusive-products",
    "https://www.sephora.com/shop/mens-perfume",
    "https://www.sephora.com/shop/mens-facial-products",
    "https://www.sephora.com/shop/mens-grooming",
    "https://www.sephora.com/shop/mens-hair-care",
    "https://www.sephora.com/shop/mens-personal-care",
    "https://www.sephora.com/beauty/new-cologne-for-men",
    "https://www.sephora.com/beauty/best-selling-mens-products"
]

In [206]:
categories = [
    "just_arrived","gifts_her", "gifts_men", "gifts_them", "gifts_teens",
    "gift_sets_men", "travel_size", "gift_sets", "editors_picks",
    "luxury_gifts", "sephora_exclusives", "perfume_men", "facial_men",
    "grooming_men", "hair_men", "personal_care_men", "just_arrived_men", "best_seller_men"
]

In [7]:
for i in categories:
    if i in df_category.columns:
        print(i)

In [126]:
def close_country_box(browser):
    try:
        browser.find_element_by_xpath("//*[@id='MediaModal']/button").click()
        return browser
    except:
        return browser


def close_login_box(browser):
    try:
        browser.find_element_by_xpath("//*[@id='modalDialog']/button").click()
        return browser
    except:
        return browser


def view_300(browser):
    try:
        if browser.find_element_by_class_name("css-1qizhe3").text.endswith("View all"):
            browser.find_element_by_class_name("css-1qizhe3").send_keys("View all")
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            scroll_to_bottom(browser, 5)
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            sleep(1)
            return browser
    except:
        return browser
        
def next_page(browser):
    try:
        if browser.find_element_by_class_name("css-4ktkov").get_attribute("aria-label")=="Next":
            browser.find_element_by_class_name("css-4ktkov").click()
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            scroll_to_bottom(browser, 5)
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            sleep(1)
            return browser
    except:
        return browser

    
def scroll_to_bottom(browser, x=20):
    sleep(1)
    close_country_box(browser)
    close_login_box(browser)
    close_country_box(browser)
    sleep(1)
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = x
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(0.5)
        close_country_box(browser)
        close_login_box(browser)
        close_country_box(browser)
        no_of_pagedowns -= 1
    return browser

In [127]:
def get_products(soup):

    prefix_url = "https://www.sephora.com"
    
    try:
        product_json = soup.find_all(attrs={"type": re.compile("text/json")})
        product_json = json.loads(product_json[0].text)
    except:
        print(filename, " - download unsuccessful")

    product_list = []

    for i in product_json:
        for key, val in i["props"].items():
            if key == "products":
                if type(val) == list:
                    for product in val:

                        product_dict = {}  #each product

                        product_dict["brand_name"] = product["brandName"]

                        product_dict["product_ref"] = product["productId"]

                        product_dict["product_item"] = product["currentSku"][
                            "skuId"]

                        product_dict["product_name"] = product["displayName"]

                        product_dict["product_price"] = product["currentSku"][
                            "listPrice"]

                        product_dict["product_image"] = prefix_url + product[
                            "currentSku"]["skuImages"]["image450"]

                        product_dict[
                            "product_link"] = prefix_url + product["targetUrl"]

                        product_list.append(product_dict)
                        
    if len(product_list)!=0:
        return product_list
    
    else:

        if len(soup.find_all("a", href=re.compile("/product/"))) == 0:
            return product_list

        else:
            for tag in soup.find_all("a", href=re.compile("/product/")):

                product_dict = {}  #each product

                product_dict["brand_name"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_brand")
                    }).get_text(strip=True).replace(" NEW", "")
                try:
                    product_dict["product_ref"] = tag.attrs["data-uid"].split()[0]
                except:
                    product_dict["product_ref"] = np.nan

                try:
                    product_dict["product_item"] = tag.attrs["data-uid"].split(
                    )[-1]
                except:
                    product_dict["product_item"] = np.nan

                try:
                    product_dict["product_name"] = tag.find(
                        attrs={
                            "data-at": re.compile("sku_item_name")
                        }).get_text(strip=True)
                except:
                    product_dict["product_name"] = np.nan

                try:
                    product_dict["product_price"] = tag.find(
                        attrs={
                            "data-at": re.compile("sku_item_price_list")
                        }).get_text(strip=True)
                except:
                    product_dict["product_price"] = np.nan

                try:
                    product_dict["product_image"] = prefix_url + tag.find(
                        attrs={
                            "src": re.compile("/productimages/")
                        }).get("src")
                    if product_dict["product_item"] is np.nan:
                        item_pattern = re.compile("s[0-9]+")
                        product_dict["product_item"] = item_pattern.findall(
                            product_dict["product_image"])[0].strip("s")
                except:
                    product_dict["product_image"] = np.nan

                try:
                    product_dict["product_link"] = prefix_url + tag.attrs["href"]
                    if product_dict["product_ref"] is np.nan:
                        ref_pattern = re.compile("P[0-9]+")
                        product_dict["product_ref"] = ref_pattern.findall(
                            product_dict["product_link"])[0]
                except:
                    product_dict["product_link"] = np.nan

                product_list.append(product_dict)

        return product_list

In [133]:
def get_page(url, page):
 
    browser.get(url)
    
    close_country_box(browser)
    close_login_box(browser)
    close_country_box(browser)
    sleep(2)
    
    scroll_to_bottom(browser, 10)
    sleep(3)
    scroll_to_bottom(browser, 10)
    sleep(3)

    view_300(browser)
    
    scroll_to_bottom(browser, 10)
    sleep(3)
    scroll_to_bottom(browser, 10)
    sleep(3)
#     scroll_to_bottom(browser, 10)
#     sleep(3)

#     if page > 1:
#         for i in range(page - 1):
#             next_page(browser)
#             scroll_to_bottom(browser)
#             sleep(3)
    
#     scroll_to_bottom(browser, 10)
#     sleep(5)
    scroll_to_bottom(browser, 10)
    sleep(10)
    
    close_country_box(browser)
    close_login_box(browser)
    close_country_box(browser)
    sleep(2)

    html = browser.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    soup.prettify()

    return soup

In [ ]:
categories_list = dict(zip(categories, urls)

In [210]:
chrome_path = "/Users/valmadrid/Downloads/chromedriver"
browser = webdriver.Chrome(executable_path=chrome_path)

for i in [-1, -2]:#tqdm(range(len(urls))):
    product_list = []
    for page in range(1, 10):
        if page == 1:
            full_url = urls[i] + "?pageSize=300"
        if page > 1:
            full_url = urls[i] + "?pageSize=300&currentPage=" + str(page)
        soup = get_page(full_url, page)
        products = get_products(soup)
        product_list.extend(products)
        if len(product_list) == 0:
            #category has 0 products, then go to next category
            break
        elif len(products) < 300:
            #current page has less than 300 items, then go to next category
            categories_list[categories[i]] = product_list
            break
        else:
            #otherwise go to next page
            sleep(2)
            continue
        sleep(randint(3, 5))
    sleep(5)

browser.quit()

In [139]:
categories_list

{'just_arrived': [{'brand_name': 'Urban Decay',
   'product_ref': 'P459819',
   'product_item': '2363588',
   'product_name': 'Naked Ultraviolet Eyeshadow Palette',
   'product_price': '$49.00',
   'product_image': 'https://www.sephora.com/productimages/sku/s2363588-main-zoom.jpg?imwidth=135&imwidth=135',
   'product_link': 'https://www.sephora.com/product/urban-decay-naked-ultraviolet-eyeshadow-palette-P459819?icid2=just arrived_us_skugrid_ufe:p459819:product'},
  {'brand_name': 'PAT McGRATH LABS',
   'product_ref': 'P458774',
   'product_item': '2351559',
   'product_name': 'Mothership VIII Artistry Eyeshadow Palette - Divine Rose II Collection',
   'product_price': '$125.00',
   'product_image': 'https://www.sephora.com/productimages/sku/s2351559-main-zoom.jpg?imwidth=135&imwidth=135',
   'product_link': 'https://www.sephora.com/product/pat-mcgrath-labs-mothership-viii-artistry-eyeshadow-palette-divine-rose-ii-P458774?icid2=just arrived_us_skugrid_ufe:p458774:product'},
  {'brand_na

In [212]:
with open("categories_list.json", "w") as outfile:
    json.dump(categories_list, outfile, indent=2)

In [213]:
for category, products_list in categories_list.items():
    check_list = [product["product_ref"] for product in list(products_list)]
    for i in range(len(df_category)):
        if df_category.id.iloc[i] in check_list:
            df_category.at[i, category] = 1

In [211]:
for k, v in categories_list.items():
    print(k)
    print(len(v))
    print(type(v) == list)

just_arrived
101
True
gifts_her
362
True
gifts_men
21
True
gifts_them
124
True
gifts_teens
58
True
gift_sets_men
33
True
travel_size
917
True
gift_sets
285
True
editors_picks
46
True
luxury_gifts
84
True
sephora_exclusives
120
True
perfume_men
229
True
facial_men
88
True
grooming_men
51
True
hair_men
50
True
personal_care_men
63
True
just_arrived_men
13
True
best_seller_men
40
True


In [214]:
df_category

,id,sheet_masks,masks,skincare,conditioner,shampoo_and_conditioner,hair,mini_size,concealer,face,makeup,rollerballs_and_travel_size,women,fragrance,value_and_gift_sets,face_masks,for_body,self_tanners,bath_and_body,moisturizers,perfume,hair_styling_products,hair_styling_and_treatments,face_wash_and_cleansers,cleansers,lip_gloss,lip,hair_straighteners_and_flat_irons,hair_tools,blush,cheek,lotions_and_oils,bath_and_shower,men,facial_cleansing_brushes,high_tech_tools,cellulite_and_stretch_marks,body_care,shampoo,eye_primer,eye,lipstick,hair_masks,perfume_gift_sets,body_lotions_and_body_oils,body_moisturizers,holistic_wellness,wellness,face_primer,eye_palettes,false_eyelashes,body_wash_and_shower_gel,lip_balm_and_treatment,scrub_and_exfoliants,eyebrow,eyeliner,exfoliators,skincare_sets,face_brushes,brushes_and_applicators,tools_and_brushes,face_serums,treatments,beauty_supplements,lip_sets,nail,tinted_moisturizer,highlighter,face_sets,makeup_removers,eye_creams_and_treatments,eye_care,face_wipes,hair_brushes_and_combs,leave_in_conditioner,mascara,foundation,body_products,other_needs,shaving,bb_and_cc_cream,hair_oil,cologne_gift_sets,dry_shampoo,bronzer,face_sunscreen,sun_care,liquid_lipstick,hair_accessories,lip_liner,cologne,lip_balms_and_treatments,lip_treatments,deodorant_and_antiperspirant,tweezers_and_eyebrow_tools,accessories,anti_aging,hair_primers,facial_peels,brush_cleaners,curling_irons,hand_cream_and_foot_cream,brush_sets,scalp_and_hair_treatments,blemish_and_acne_treatments,eye_cream,toners,sponges_and_applicators,setting_spray_and_powder,moisturizer_and_treatments,face_oils,eye_brushes,facial_rollers,face_wash,candles,candles_and_home_scents,body_mist_and_hair_mist,makeup_palettes,value_sets,mists_and_essences,color_correct,eyeshadow,body_sprays_and_deodorant,aftershave,hair_dryers,hair_spray,contour,makeup_brushes_and_applicators,eye_sets,body_sunscreen,eyelash_curlers,night_creams,decollete_and_neck_creams,sunscreen,mirrors_and_sharpeners,eye_masks,hair_thinning_and_hair_loss,teeth_whitening,lip_brushes,cheek_palettes,bath_soaks_and_bubble_bath,makeup_bags_and_travel_cases,for_face,blotting_papers,color_care,best_for,hair_products,bb_and_cc_creams,lip_plumper,lip_stain,cleansing_brushes,deodorant_for_men,hand_wash_and_soap,hair_removal,diffusers,hair_removal_and_shaving,curls_and_coils,makeup_and_travel_cases,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,travel_size,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,facial_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men
0,P420666,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,P448857,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,P425464,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,P428606,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [216]:
df_category.sephora_exclusives.value_counts()

1.00    73
Name: sephora_exclusives, dtype: int64